<a href="https://colab.research.google.com/github/marued/low-resource-machine-translation-team07/blob/master/notebooks/Unsupervised_Neural_Machine_Translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Unsupervised Neural Machine Translation
reproduce the result on our corpus, 
Project: https://github.com/artetxem/undreamt


In [0]:
import os
os.chdir("/home/ryan/projects/undreamt/data")
os.listdir()

['en_bpe.model',
 'train.lang1',
 'en_corpus.txt',
 'unaligned.fr',
 'train.lang2',
 'fr_corpus.txt.atok',
 'en_corpus.txt.atok',
 'unaligned.en',
 'fr_corpus.txt',
 'en_bpe.vocab',
 'fr_bpe.model',
 'restult_dimension_300',
 'fr_bpe.vocab']

In [7]:
!pip install sentencepiece

  Cache entry deserialization failed, entry ignored
  Using cached https://files.pythonhosted.org/packages/74/f4/2d5214cbf13d06e7cb2c20d84115ca25b53ea76fa1f0ade0e3c9749de214/sentencepiece-0.1.85-cp36-cp36m-manylinux1_x86_64.whl


In [0]:
import sentencepiece as spm

## Train sentencpiece 

In [0]:
# merge two corpus
! cat train.lang1 unaligned.en > en_corpus.txt
! cat train.lang2 unaligned.fr > fr_corpus.txt

In [0]:
# train sentencepiece model from `botchan.txt` and makes `m.model` and `m.vocab`
# `m.vocab` is just a reference. not used in the segmentation.
spm.SentencePieceTrainer.train('--input=en_corpus.txt --model_prefix=en_bpe --vocab_size=10000 --model_type=bpe')
spm.SentencePieceTrainer.train('--input=fr_corpus.txt --model_prefix=fr_bpe --vocab_size=10000 --model_type=bpe')


True

In [12]:
# Test
# makes segmenter instance and loads the model file
sp_en = spm.SentencePieceProcessor()
sp_en.load('en_bpe.model')

# encode: text => id
sentence="Mr President, it seems that the long road to creating a single regulation on the legal protection of designs, drawings and models in the European Union is nearing its end."
print(sp_en.encode_as_pieces(sentence))
print(sp_en.encode_as_ids(sentence))

encoded_sentence = sp_en.encode_as_pieces(sentence)
print(sp_en.decode_pieces(encoded_sentence))

['▁Mr', '▁President', ',', '▁it', '▁seems', '▁that', '▁the', '▁long', '▁road', '▁to', '▁creating', '▁a', '▁single', '▁regulation', '▁on', '▁the', '▁legal', '▁protection', '▁of', '▁design', 's', ',', '▁draw', 'ings', '▁and', '▁models', '▁in', '▁the', '▁European', '▁Union', '▁is', '▁near', 'ing', '▁its', '▁end', '.']
[199, 268, 9950, 74, 1719, 51, 8, 609, 2061, 23, 2751, 5, 1592, 1643, 62, 8, 1193, 1191, 29, 1911, 9934, 9950, 1876, 484, 40, 5342, 28, 8, 166, 351, 49, 2475, 34, 302, 672, 9948]
Mr President, it seems that the long road to creating a single regulation on the legal protection of designs, drawings and models in the European Union is nearing its end.


In [0]:
# Test
# makes segmenter instance and loads the model file
sp_fr = spm.SentencePieceProcessor()
sp_fr.load('fr_bpe.model')

# encode: text => id
sentence="Les élections palestiniennes de janvier ont été remportées par un leader relativement modéré, et ce pour deux raisons : il a été mis en avant par le mouvement Fatah au pouvoir et les Islamistes ont boycotté l'élection."
print(sp_fr.encode_as_pieces(sentence))
print(sp_fr.encode_as_ids(sentence))

['▁Les', '▁élections', '▁palestini', 'ennes', '▁de', '▁janvier', '▁ont', '▁été', '▁remp', 'or', 'tées', '▁par', '▁un', '▁leader', '▁relativement', '▁modé', 'ré', ',', '▁et', '▁ce', '▁pour', '▁deux', '▁raisons', '▁:', '▁il', '▁a', '▁été', '▁mis', '▁en', '▁avant', '▁par', '▁le', '▁mouvement', '▁F', 'at', 'ah', '▁au', '▁pouvoir', '▁et', '▁les', '▁Is', 'lam', 'istes', '▁ont', '▁b', 'oy', 'co', 't', 'té', '▁l', "'", 'élection', '.']
[292, 2062, 3513, 2462, 11, 3955, 248, 284, 1802, 35, 1881, 74, 51, 8414, 4600, 6327, 119, 9933, 41, 60, 78, 429, 1739, 671, 160, 16, 284, 1313, 55, 775, 74, 36, 3655, 544, 109, 3051, 66, 897, 41, 45, 9538, 3897, 1951, 248, 85, 329, 372, 9917, 80, 5, 9932, 5872, 9931]


In [0]:
sp_en = spm.SentencePieceProcessor()
sp_en.load('en_bpe.model')

with open("en_corpus.txt", 'r') as f_in, open("en_corpus.txt.atok", 'w') as f_out:
    for line in f_in:
        f_out.writelines(" ".join(sp_en.encode_as_pieces(line)+ ['\n']))

In [0]:
sp_fr = spm.SentencePieceProcessor()
sp_fr.load('fr_bpe.model')

with open("fr_corpus.txt", 'r') as f_in, open("fr_corpus.txt.atok", 'w') as f_out:
    for line in f_in:
        f_out.writelines(" ".join(sp_fr.encode_as_pieces(line)+ ['\n']))

## Train word embedding

In [0]:
!/home/ryan/projects/fastText/fasttext skipgram -input en_corpus.txt.atok -output restult_dimension_300/fasttext-en-300 -minn 2 -maxn 5 -dim 300

Read 11M words
Number of words:  9902
Number of labels: 0
Progress: 100.0% words/sec/thread:   29057 lr:  0.000000 avg.loss:  2.268167 ETA:   0h 0m 0s


In [0]:
!/home/ryan/projects/fastText/fasttext skipgram -input fr_corpus.txt.atok -output restult_dimension_300/fasttext-fr-300 -minn 2 -maxn 5 -dim 300

Read 13M words
Number of words:  9921
Number of labels: 0
Progress: 100.0% words/sec/thread:   27845 lr:  0.000000 avg.loss:  2.274154 ETA:   0h 0m 0s


Mapping two languange word embedding to same vector space

In [0]:
!pip install cupy-cuda102

In [0]:
!python /home/ryan/projects/vecmap/map_embeddings.py --unsupervised restult_dimension_300/fasttext-en-300.vec restult_dimension_300/fasttext-fr-300.vec SRC_MAPPED.EMB TRG_MAPPED.EMB --cuda

## Train NMT model

In [2]:
import os
os.chdir("/home/ryan/projects/undreamt")
os.listdir('data')

['en_bpe.model',
 'train.lang1',
 'en_corpus.txt',
 'test_en.txt.atok',
 'unaligned.fr',
 'test_fr.txt.atok',
 'train.lang2',
 'fr_pred.txt.atok',
 'fr_corpus.txt.atok',
 'en_corpus.txt.atok',
 'unaligned.en',
 'fr_corpus.txt',
 'en_bpe.vocab',
 'fr_bpe.model',
 'restult_dimension_300',
 'fr_bpe.vocab']

In [0]:
!pip install torch==0.3.1

In [0]:
!python train.py --src data/en_corpus.txt.atok --trg data/fr_corpus.txt.atok --src_embeddings data/restult_dimension_300/SRC_MAPPED.EMB --trg_embeddings data/restult_dimension_300/TRG_MAPPED.EMB --save lowresource_enfr --cuda

In [18]:
!python3 translate.py lowresource_enfr.final.src2trg.pth < data/test_en.txt.atok > data/test_fr_pred.txt.atok

/home/ryan/projects/undreamt/undreamt/encoder.py:48: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters().
  output, hidden = self.rnn(embeddings, hidden)


Get the BLEU score before decoding

In [28]:
 !perl multi-bleu.perl data/test_fr.txt.atok < data/test_fr_pred.txt.atok 

BLEU = 15.23, 44.6/20.9/11.6/6.7 (BP=0.928, ratio=0.930, hyp_len=26178, ref_len=28135)


In [0]:
sp_fr = spm.SentencePieceProcessor()
sp_fr.load('data/fr_bpe.model')

In [0]:
with open("data/test_fr_pred.txt.atok", 'r') as f_in, open("data/test_fr_pred.txt", 'w') as f_out:
    for line in f_in:
        f_out.writelines([sp_fr.decode_pieces(line.split()), '\n'])

After decoding the BPE text.

In [30]:
!perl multi-bleu.perl data/test_fr.txt < data/test_fr_pred.txt

BLEU = 15.40, 46.5/20.8/11.2/6.3 (BP=0.952, ratio=0.953, hyp_len=22258, ref_len=23353)


In [1]:
import sys
print(sys.executable)

/home/ryan/projects/undreamt/unsup_nmt_env/bin/python3.6
